### ruijin data

In [54]:
label = '/remote-home/share/RJ_video/RJ_video_crop/final_label.csv'

In [55]:
import pandas as pd
df = pd.read_csv(label,dtype=str)

In [56]:
df['ID'] = df['ID'].str.zfill(9)

In [58]:
df.to_excel('./videoid.xlsx')

In [3]:
import os
root = '/remote-home/share/RJ_video/RJ_video_crop'

In [7]:
pcnt = 0
v_cnt = 0
img_cnt = 0
for p in os.listdir(root):
    p_path = os.path.join(root,p)
    if os.path.isdir(p_path):
        pcnt += 1
        for f in os.listdir(p_path):
            if f.endswith('.mp4') or f.endswith('.avi'):
                v_cnt += 1
            else:
                img_cnt += 1
print(pcnt)
print(v_cnt)
print(img_cnt)

1196
1221
2579


# Multi-Modal

In [11]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import torch

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [12]:
from Dataset.UEset import UEset
from models.vgg import vgg16_bn_tiny
from models.resnet import ResNet10

In [10]:
pre_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    ])
root = '/remote-home/share/MM_Ultrasound'

In [4]:
data = UEset(root, pre_transform = pre_transform, sub_list = [1,2,3,4,5],label=True)

In [5]:
len(data)

3474

In [6]:
loader = DataLoader(data,batch_size=256,shuffle=True,num_workers=4)

In [32]:
input = torch.randn((128,4,3,224,224))

In [36]:
f11 = output[:,0,:]

In [37]:
f11.shape

torch.Size([128, 500])

In [5]:
imgs, label = data[19]

In [9]:
img11, img21, img12, img22 = imgs

In [11]:
img11.shape

torch.Size([3, 224, 224])

In [13]:
model = vgg16_bn_tiny()

In [22]:
output = model(img11.unsqueeze(0))

In [25]:
len(output)

5

In [27]:
model2 = ResNet10()

In [29]:
model2(img11.unsqueeze(0)).shape

torch.Size([1, 512, 7, 7])

## For InfoNCE

In [38]:
# 256 imgs with dim 512
f11 = torch.randn(256,512)
f21 = torch.randn(256,512)
f12 = torch.randn(256,512)
f22 = torch.randn(256,512)

In [44]:
f11.size(0)

256

In [39]:
criterion = nn.CrossEntropyLoss()

In [41]:
t = 0.01
# for data augmentation loss
l_pos_aug1 = torch.einsum('nc,nc->n',[f11,f12]).unsqueeze(-1)
l_pos_aug2 = torch.einsum('nc,nc->n',[f21,f22]).unsqueeze(-1)
l_neg_aug1 = torch.einsum('nc,ck->nk',[f11,f12.T])
l_neg_aug2 = torch.einsum('nc,ck->nk',[f21,f22.T])
logits_aug1 = torch.cat([l_pos_aug1,l_neg_aug1],dim=1)
logits_aug2 = torch.cat([l_pos_aug2,l_neg_aug2],dim=1)
labels = torch.zeros(logits_aug1.shape[0],dtype=torch.long)
aug_loss = criterion(logits_aug1,labels) + criterion(logits_aug2,labels)


In [42]:
labels.shape

torch.Size([256])

In [47]:
# for domain invariant loss
l_pos_d1 = torch.einsum('nc,nc->n',[f11,f21]).unsqueeze(-1)
l_pos_d2 = torch.einsum('nc,nc->n',[f12,f22]).unsqueeze(-1)
l_neg_d1 = torch.einsum('nc,ck->nk',[f11,f21.T])
l_neg_d2 = torch.einsum('nc,ck->nk',[f12,f22.T])
logits_d1 = torch.cat([l_pos_d1,l_neg_d1],dim=1)
logits_d2 = torch.cat([l_pos_d2,l_neg_d2],dim=1)
labels = torch.zeros(logits_d1.shape[0],dtype=torch.long)
d_loss = criterion(logits_d1,labels) + criterion(logits_d2,labels)

In [48]:
aug_loss

tensor(127.3748)

In [49]:
d_loss

tensor(130.3878)

## draft for prediction

In [29]:
prob = torch.randn(5,2)

In [30]:
prob

tensor([[-0.8591, -0.8592],
        [ 0.1835,  1.0775],
        [ 0.6239, -1.1920],
        [-0.9117, -0.7077],
        [-0.5845, -0.5312]])

In [31]:
a = torch.argmax(prob,dim=1)

In [38]:
a

tensor([0, 1, 0, 1, 1])

In [36]:
softmax_prob = torch.softmax(prob,dim=1)

In [37]:
softmax_prob

tensor([[0.5000, 0.5000],
        [0.2903, 0.7097],
        [0.8601, 0.1399],
        [0.4492, 0.5508],
        [0.4867, 0.5133]])

In [70]:
prob = softmax_prob[:,1]

In [71]:
prob

tensor([0.5000, 0.7097, 0.1399, 0.5508, 0.5133])

In [42]:
prob, pred = torch.max(softmax_prob,dim=1)

In [68]:
prob

tensor([0.5000, 0.7097, 0.8601, 0.5508, 0.5133])

In [69]:
pred

tensor([0, 1, 0, 1, 1])

In [52]:
p=[]

In [55]:
p.extend(prob.cpu().detach().tolist())

In [56]:
p

[0.5000395774841309,
 0.7097166776657104,
 0.8600674867630005,
 0.5508385896682739,
 0.5133243203163147,
 0.5000395774841309,
 0.7097166776657104,
 0.8600674867630005,
 0.5508385896682739,
 0.5133243203163147]

In [49]:
p.append(prob.cpu().detach().numpy().ravel())

In [50]:
p

[array([0.5000396, 0.7097167, 0.8600675, 0.5508386, 0.5133243],
       dtype=float32),
 array([0.5000396, 0.7097167, 0.8600675, 0.5508386, 0.5133243],
       dtype=float32)]

In [45]:
prob.cpu().detach().numpy().ravel()

array([0.5000396, 0.7097167, 0.8600675, 0.5508386, 0.5133243],
      dtype=float32)

In [44]:
pred

tensor([0, 1, 0, 1, 1])

In [32]:
a.tolist()

[0, 1, 0, 1, 1]

In [28]:
label = [1,1,1,1,1]

In [27]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, accuracy_score

In [34]:
classification_report(label,a.tolist(),output_dict = True, zero_division=0)

{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0},
 '1': {'precision': 1.0,
  'recall': 0.6,
  'f1-score': 0.7499999999999999,
  'support': 5},
 'accuracy': 0.6,
 'macro avg': {'precision': 0.5,
  'recall': 0.3,
  'f1-score': 0.37499999999999994,
  'support': 5},
 'weighted avg': {'precision': 1.0,
  'recall': 0.6,
  'f1-score': 0.7499999999999999,
  'support': 5}}

## test for vgg encoder

In [16]:
from models.vgg import vgg16_bn_tiny
from models.encoder import Vgg_Encoder
encoder = Vgg_Encoder()

In [17]:
input = torch.randn(16,4,3,224,224)
a = encoder(input)

In [23]:
batch, flist, vector = a

In [24]:
vector.shape

torch.Size([16, 4, 500])

In [25]:
for f in flist:
    print(f.shape)

torch.Size([64, 16, 224, 224])
torch.Size([64, 32, 112, 112])
torch.Size([64, 64, 56, 56])
torch.Size([64, 128, 28, 28])
torch.Size([64, 128, 14, 14])


## for decoder

In [10]:
import numpy as np
def l2_normal(x,dim=1):
    norm = x.pow(2).sum(dim, keepdim=True).pow(1./2)+1e-10
    out = x.div(norm)
    return out
            
def l1_normal(x,dim=1):
    x=x+10e-10
    norm = x.sum(dim, keepdim=True)
    out = x.div(norm)
    return out  

class decoder(nn.Module):
    def __init__(self,channel_list=[128,128,64,32,16,32,8]):
        super(decoder, self).__init__()
        self.upsample=nn.Upsample(scale_factor=(2,2))
        
        
        self.conv_layer8x=nn.Sequential(
                                              nn.Conv2d(in_channels=channel_list[0]+channel_list[1],out_channels=channel_list[1],kernel_size=(3,3),padding=(1,1)),
                                              nn.BatchNorm2d(channel_list[1]),
                                              nn.ReLU(True),
                                              nn.Conv2d(in_channels=channel_list[1],out_channels=channel_list[2],kernel_size=(3,3),padding=(1,1)),
                                              nn.BatchNorm2d(channel_list[2]),
                                              nn.ReLU(True))

        self.conv_layer4x=nn.Sequential(
                                              nn.Conv2d(in_channels=channel_list[2]*2,out_channels=channel_list[2],kernel_size=(3,3),padding=(1,1)),
                                              nn.BatchNorm2d(channel_list[2]),
                                              nn.ReLU(True),
                                              nn.Conv2d(in_channels=channel_list[2],out_channels=channel_list[3],kernel_size=(3,3),padding=(1,1)),
                                              nn.BatchNorm2d(channel_list[3]),
                                              nn.ReLU(True)
                                                )
        self.conv_layer2x=nn.Sequential(
                                              nn.Conv2d(in_channels=channel_list[3]*2,out_channels=channel_list[3],kernel_size=(3,3),padding=(1,1)),
                                              nn.BatchNorm2d(channel_list[3]),
                                              nn.ReLU(True),
                                              nn.Conv2d(in_channels=channel_list[3],out_channels=channel_list[4],kernel_size=(3,3),padding=(1,1)),
                                              nn.BatchNorm2d(channel_list[4]),
                                              nn.ReLU(True)
                                                )
        self.embedding_branch=nn.Sequential(
                                              nn.Conv2d(in_channels=channel_list[4]*2,out_channels=channel_list[5],kernel_size=(3,3),padding=(1,1)),
                                              nn.BatchNorm2d(channel_list[5]),
                                              nn.ReLU(True),
                                              nn.Conv2d(in_channels=channel_list[5],out_channels=channel_list[5],kernel_size=(3,3),padding=(1,1))
                                                )
        if len(channel_list)>6:
            self.clustering_branch=nn.Sequential(
                                                  nn.Conv2d(in_channels=channel_list[4]*2,out_channels=channel_list[6],kernel_size=(3,3),padding=(1,1)),
                                                  nn.BatchNorm2d(channel_list[6]),
                                                  nn.ReLU(True),
                                                  nn.Conv2d(in_channels=channel_list[6],out_channels=channel_list[6],kernel_size=(3,3),padding=(1,1)),
                                                  nn.Sigmoid()
                                                )
    def forward(self, x,region_output_size=(4,4),if_clustering=False,if_test=False):
        features=self.upsample(x[-1])
        features=torch.cat([features,x[-2]],dim=1)
        features=self.conv_layer8x(features)
        features=self.upsample(features)
        features=torch.cat([features,x[-3]],dim=1)
        features=self.conv_layer4x(features)
        features=self.upsample(features)
        features=torch.cat([features,x[-4]],dim=1)
        features=self.conv_layer2x(features)
        features=self.upsample(features)
        features=torch.cat([features,x[-5]],dim=1)
        del x
        if not if_clustering:
            features=self.embedding_branch(features)   # embedding of each pixel
            features=l2_normal(features)  # ||pixel feature||_2 = 1
            return features
        
        attention_map=self.clustering_branch(features)
        attention_map=l1_normal(attention_map, dim=1)
        features=self.embedding_branch(features)
        
        features=l2_normal(features)
        # return attention_map,features
        if if_test:
            return attention_map,features
        patch_features=nn.functional.adaptive_avg_pool2d(features,output_size=region_output_size)
        print(features.shape)
        region_features=features[:,np.newaxis,:,:,:]*attention_map[:,:,np.newaxis,:,:]
        region_features=torch.mean(region_features,(3,4))
        region_features=l2_normal(region_features,2)
        return l2_normal(patch_features),region_features,attention_map

In [14]:
de = decoder()

### Understand Decoder

In [42]:
attns, fs =de(a, if_clustering=True, if_test=True)

In [43]:
print(attns.shape)
print(fs.shape)

torch.Size([2, 8, 224, 224])
torch.Size([2, 32, 224, 224])


In [48]:
patch, region, attn_map =de(a, if_clustering=True)

torch.Size([2, 32, 224, 224])


In [45]:
print(patch.shape)
print(region.shape)
print(attn_map.shape)

torch.Size([2, 32, 4, 4])
torch.Size([2, 8, 32])
torch.Size([2, 8, 224, 224])


In [40]:
region.shape

torch.Size([2, 8, 32])

In [41]:
attn_map.shape

torch.Size([2, 8, 224, 224])

In [24]:
f = de(a)

torch.Size([2, 128, 28, 28])


In [27]:
f.shape

torch.Size([2, 32, 224, 224])

In [28]:
fs.shape

torch.Size([2, 32, 224, 224])

torch.Size([2, 8, 224, 224])

In [49]:
bb = torch.tensor(1.0)

In [56]:
bb.cpu().item()

1.0

### overall encoder and decoder

In [15]:
imgs = torch.randn(16,3,224,224)

In [16]:
f = encoder(imgs)

In [26]:
patch, region, attn_map =de(f, region_output_size=(8,8), if_clustering=True)

torch.Size([16, 32, 224, 224])


In [27]:
patch.shape

torch.Size([16, 32, 8, 8])

### look into patch features

In [118]:
import torch.nn.functional as F
class Dense_Loss(nn.Module):
    def __init__(self,patch_size,h=[0,1,2,3,4]):
        super(Dense_Loss, self).__init__()
        self.h = h
        self.patch_size = patch_size

    def forward(self, f):
        all_loss = torch.tensor(0.)
        for i in self.h:
            feature_map = f[i]
            feature_map = F.normalize(feature_map,dim=1) ## l2 norm first
            print(feature_map.shape)
            b,c,w,h = feature_map.shape
            feature_map = feature_map.view(-1,4,c,w,h) # reshape the feature map
            fm11 = feature_map[:,0,:,:]
            fm21 = feature_map[:,1,:,:]
            print(fm11.shape)
            patch11 = F.adaptive_avg_pool2d(fm11,output_size=self.patch_size)
            patch21 = F.adaptive_avg_pool2d(fm21,output_size=self.patch_size)
            print(patch11.shape)
            # now the feature map in each modality is b x c x n x n (n denotes patch size)
            print(b)
            print(c)
            patch_flatten11 = patch11.view(int(b/4),c,-1)
            patch_flatten21 = patch21.view(int(b/4),c,-1)
            # b x c x n^2
            sim_matrix11 = torch.einsum('abc,abd->acd',[patch_flatten11,patch_flatten11]) # b x n^2
            sim_matrix21 = torch.einsum('abc,abd->acd',[patch_flatten21,patch_flatten21])
            sim_flatten11 = sim_matrix11.view(sim_matrix11.shape[0],-1)
            sim_flatten21 = sim_matrix21.view(sim_matrix21.shape[0],-1)
            patch_loss = F.pairwise_distance(sim_flatten11,sim_flatten21,p=2)
            h_loss = torch.sum(patch_loss)
            all_loss += h_loss
        return all_loss
            
            

In [122]:
d_loss = Dense_Loss(patch_size=[4,4],h=[0,1,2,3,4])

In [123]:
loss = d_loss(flist)

torch.Size([64, 16, 224, 224])
torch.Size([16, 16, 224, 224])
torch.Size([16, 16, 4, 4])
64
16
torch.Size([64, 32, 112, 112])
torch.Size([16, 32, 112, 112])
torch.Size([16, 32, 4, 4])
64
32
torch.Size([64, 64, 56, 56])
torch.Size([16, 64, 56, 56])
torch.Size([16, 64, 4, 4])
64
64
torch.Size([64, 128, 28, 28])
torch.Size([16, 128, 28, 28])
torch.Size([16, 128, 4, 4])
64
128
torch.Size([64, 128, 14, 14])
torch.Size([16, 128, 14, 14])
torch.Size([16, 128, 4, 4])
64
128


In [124]:
loss

tensor(6.7840, grad_fn=<AddBackward0>)

In [112]:
sim_flatten11.shape

torch.Size([16, 256])

In [113]:
output = F.pairwise_distance(sim_flatten11,sim_flatten21,p=2)

In [115]:
torch.sum(output)

tensor(0.4395, grad_fn=<SumBackward0>)

In [28]:
flatten_patch = patch.view(-1,32,8*8)

In [36]:
flatten_patch.shape

torch.Size([16, 32, 64])

In [34]:
sim_matrix = torch.einsum('abc,abd->acd',[flatten_patch,flatten_patch])

In [35]:
sim_matrix.shape

torch.Size([16, 64, 64])

In [19]:
patch = patch.view(-1,4,32,8,8)

In [20]:
patch.shape

torch.Size([4, 4, 32, 8, 8])

In [21]:
map11 = patch[:,1,:,:,:]
map21 = patch[:,2,:,:,:]

In [22]:
map11.shape

torch.Size([4, 32, 8, 8])

In [80]:
sim_matrix = torch.einsum('abcd,abcd->acd',[map11,map21])

In [81]:
sim_matrix.shape

torch.Size([4, 8, 8])

### feature maps in encoder

In [24]:
for fmap in f:
    print(fmap.shape)

torch.Size([16, 16, 224, 224])
torch.Size([16, 32, 112, 112])
torch.Size([16, 64, 56, 56])
torch.Size([16, 128, 28, 28])
torch.Size([16, 128, 14, 14])
